<a href="https://colab.research.google.com/github/DLSeed/DeepLearning/blob/main/%E7%B0%A1%E5%8D%98%E3%83%97%E3%83%AD%E3%83%B3%E3%83%97%E3%83%88%E3%82%A2%E3%83%8B%E3%83%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 簡単プロンプトアニメ [[概要と作例はこちら](https://twitter.com/Zuntan03/status/1703674198101803268)]<br>1. ↓の ▶ で入力欄を出して、プロンプトを入力して、2. の ▶ で動画を生成します。
# @markdown **← の ▶ を押すとプロンプト入力欄が表示されます。**<br>
# @markdown [AnimateDiff prompt travel](https://github.com/s9roll7/animatediff-cli-prompt-travel) でプロンプトから簡単に動画を生成できます。<br>
# @markdown 必要なのは Google アカウントだけで、スマホからでも動画を生成できます。<br>
# @markdown 「[簡単プロンプトアニメ用の簡単LECO](https://colab.research.google.com/drive/1LYi2kvZV1XN05MATqhqWs79cNdTkb33P)」でプロンプトアニメ用の LECO を作成できます。
# @markdown **生成した動画は [Flowframes](https://nmkd.itch.io/flowframes) などでフレームレートをあげるとヌルヌルします。**
textarea_default = """# フレーム数＋コロン(:)＋プロンプトで、アニメのキーフレームにプロンプトを指定します。
# フレーム数＋コロンではなく、「H:」「F:」「N:」「L:」の場合はアニメ全体に特別な設定をします。
# シャープ(#)以降はコメントです。

H:1girl
# 「H:」か「h:」で始まる行はヘッダプロンプトの指定で、アニメの全フレームでプロンプトの先頭にヘッダプロンプト足します。

0:standing
10:dancing
20:stylish pose, explosion
# アニメの 0フレームに「standing」、10フレームに「dancing」、20フレームに「stylish pose,explosion」のプロンプトを使用します。
# デフォルトでは 1秒間に 10枚の画像が表示され(10 Frame per second)で、プロンプトを指定したキーフレームの間は補間されます。

F:maid outfit
# 「F:」か「f:」で始まる行はフッタプロンプトの指定で、アニメの全フレームでプロンプトの末尾にフッタプロンプトを足します。

# 上記の指定でキーフレームのプロンプトは以下になります。
#  0フレーム「1girl, standing, maid outfit」
# 10フレーム「1girl, dancing, maid outfit」
# 20フレーム「1girl, stylish pose, explosion, maid outfit」

N:(worst quality, low quality:1.4)
#「N:」か「n:」で始まる行は、アニメの全フレームで使用するネガティブプロンプトを指定します。

# L:explosion:-0.2
#「L:」か「l:」で始まる行は、LoRA のファイル名＋コロン(:)＋強度を、カンマ(,)区切りで複数指定できます。
# LoRAを使いたい場合は、Google Driveの `EasyPromptAnime/data/lora/` に *.safetensors 形式の LoRA を置いてください。
# 使える LoRA は通常の LierLa 形式で、C3Lier(Locon) 以降は使えないっぽいです。
"""

import ipywidgets as widgets
from IPython.display import display
textarea = widgets.Textarea(value=textarea_default, layout=widgets.Layout(width='95%', height='400px'))
display(textarea)

In [ ]:
import os, shutil, gc, time
total_start_time = time.time()
from google.colab import drive, files, runtime
from datetime import datetime
from pytz import timezone

# @title 2. ↓の ▶ でプロンプトから動画を生成します。プロンプトの保存用にGoogleドライブへのアクセスを許可してください。

# @markdown 有効にすると生成した動画を自動でダウンロードします。
movie_download = True # @param {type:"boolean"}

# @markdown GoogleDrive に生成した動画や画像を保存します。
# @markdown `All`: 動画と画像、`Movie`: 動画のみ、`Disable`: 保存しません
output_to_google_drive = "Movie" # @param ["All", "Movie", "Disable"]

# @markdown 動画の総フレーム数（画像数）です。1秒 10フレームで 30フレームなら 3秒の動画になります。
length = 30 # @param { type:"slider", min:10, max:200, step:10 }
# @markdown 生成する動画の幅です。面積は生成時間に比例し、大きくしすぎるとメモリ不足のエラーになります。
width = 384  # @param { type:"slider", min:256, max:763, step:32 }
# @markdown 生成する動画の高さです。面積は生成時間に比例し、大きくしすぎるとメモリ不足のエラーになります。
height = 512  # @param { type:"slider", min:256, max:763, step:32 }
# @markdown アップスケール方法を指定します。`Refine` は `Tile` よりきれいになりますが、メモリ不足のエラーが起きやすくなります。
upscale_mode = "Tile" # @param ["Tile", "Refine", "Disable"]
# @markdown アップスケール後の大きさを、動画の幅で指定します。面積は生成時間に比例し、大きくしすぎるとエラーになります。
# @markdown 最大値は `W384 x H512` で `Refine: 640` あたり, `Tile: 896` あたり
upscale_width = 640  # @param { type:"slider", min:384, max:1024, step:32 }
# @markdown 生成時のシードです。
seed = -1 # @param {type:"integer"}
# @markdown 動画を指定した回数生成します。複数回の生成ではシードを -1 にしてください。
count = 1 # @param { type:"slider", min:1, max:30 }

# @markdown 生成に使用するモデルを選択します。リストにないモデルを使う場合は `Custom` にして Google Drive にモデルを置きます。
model_name = "nadenadesitai_v10" # @param ["mistoonAnime_v20", "nadenadesitai_v10", "onigiriMix_v10", "surisurisitai_v10", "xxmix9realistic_v40", "Custom"]
# @markdown Custom 指定時のモデルのファイル名です。Google Drive の `EasyPromptAnime/data/models/sd/` に `*.safetensors`形式のモデルを置いてください。[stable-diffusion-model-toolkit](https://github.com/arenasys/stable-diffusion-webui-model-toolkit) などで VAE をモデルに組み込んでください。
custom_model_name = "onigiriMix_v10" # @param { type:"string" }

# @markdown ----
# @markdown **以下は詳細設定ですので、操作の必要はありません。**
# @markdown ## 動画
context = 16 # @param { type:"slider", min:4, max:32 }
motion_module = "mm_sd_v15_v2" # @param ["mm_sd_v15_v2", "mm_sd_v15", "mm_sd_v14"]

output_fps = 10 # @param { type:"slider", min:1, max:60 }
output_crf = 20 # @param { type:"slider", min:10, max:30 }

# @markdown ## 画像
scheduler = "k_dpmpp_sde" # @param ["k_dpmpp_sde", "ddim", "euler", "euler_a"]
steps = 20 # @param { type:"slider", min:10, max:50 }
guidance_scale = 8 # @param { type:"slider", min:0.1, max:30, step:0.1 }
clip_skip = 2 # @param [1, 2] {type:"raw"}

# @markdown ## アップスケール
upscale_scheduler = "k_dpmpp_sde" # @param ["k_dpmpp_sde", "ddim", "euler", "euler_a"]
upscale_steps = 20 # @param { type:"slider", min:10, max:50 }
upscale_guidance_scale = 8 # @param { type:"slider", min:0.1, max:30, step:0.1 }
upscale_strength = 0.6 # @param { type:"slider", min:0.0, max:1.0, step:0.05 }

def generate_config(name):
  return f"""{{
  "name": "{name}",
  "path": "{model_path}",
  "motion_module": "{motion_module_path}",
  "compile": false,
  "seed": [{seed}],
  "scheduler": "{scheduler}",
  "steps": {steps},
  "guidance_scale": {guidance_scale},
  "clip_skip": {clip_skip},
  "head_prompt": "{header_prompt}",
  "prompt_map": {{
{prompt_map}
  }},
  "tail_prompt": "{footer_prompt}",
  "n_prompt": ["{negative_prompt}"],
  "lora_map": {{
{lora_map}
  }},
  "upscale_config": {{
    "scheduler": "{upscale_scheduler}",
    "steps": {upscale_steps},
    "strength": {upscale_strength},
    "upscale_guidance_scale": {upscale_guidance_scale},
    "controlnet_tile": {{
      "enable": true,
      "use_preprocessor": false,
      "controlnet_conditioning_scale": 1.0,
      "guess_mode": false,
      "control_guidance_start": 0.0,
      "control_guidance_end": 1.0
    }}
  }},
  "output": {{
    "format": "mp4",
    "fps": {output_fps},
    "encode_param": {{
      "crf": {output_crf}
    }}
  }}
}}
"""

# Perse textarea
header_prompt = ""
footer_prompt = ""
negative_prompt = ""
lora_map = ""
prompt_map = ""
for line in textarea.value.splitlines():
  strip_line = line.split("#", 1)[0].strip()
  if strip_line == "":
    continue
  line_kvp = strip_line.split(":", 1)
  if len(line_kvp) < 2:
    print(f"不正な行: \"{line}\"")
    continue
  line_key = line_kvp[0].strip().lower()
  line_value = line_kvp[1].strip()
  if line_key == "h":
    if header_prompt != "":
      header_prompt += ", "
    header_prompt += line_value
  elif line_key == "f":
    if footer_prompt != "":
      footer_prompt += ", "
    footer_prompt += line_value
  elif line_key == "n":
    if negative_prompt != "":
      negative_prompt += ", "
    negative_prompt += line_value
  elif line_key == "l":
    for lora_def in line_value.split(","):
      lora_kvp = lora_def.split(":", 1)
      if len(lora_kvp) < 2:
        print(f"不正なLoRA定義: \"{lora_def}\"")
        continue
      if lora_map != "":
        lora_map += ",\n"
      lora_key = lora_kvp[0].strip()
      lora_value = lora_kvp[1].strip()
      lora_map += f"    \"lora/{lora_key}.safetensors\": {lora_value}"
  elif line_key.isdigit():
    if prompt_map != "":
      prompt_map += ",\n"
    prompt_map += f"    \"{line_key}\": \"{line_value}\""
  else:
    print(f"不正な行: \"{line}\"")
    continue

# Initialize
initialize_start_time = time.time()
data_dir = "data"
lora_dir = data_dir + "/lora"
embeddings_dir = data_dir + "/embeddings"

gd_path = "GoogleDrive/MyDrive/EasyPromptAnime"
gd_model_dir = gd_path + "/data/models/sd"
gd_lora_dir = gd_path + "/data/lora"
gd_embeddings_dir = gd_path + "/data/embeddings"
gd_config_dir = gd_path + "/config"
gd_output_dir = gd_path + "/output"
gd_upscaled_dir = gd_path + "/upscaled"
gd_refine_dir = gd_path + "/refine"
gd_movie_dir = gd_path + "/movie"

base_path = "/content/animatediff-cli-prompt-travel"
pwd = %pwd
if pwd != base_path:
  !git clone https://github.com/s9roll7/animatediff-cli-prompt-travel
  %cd {base_path}
  drive.mount("GoogleDrive")
  os.makedirs(gd_model_dir, exist_ok=True)
  os.makedirs(gd_lora_dir, exist_ok=True)
  os.makedirs(gd_embeddings_dir, exist_ok=True)
  os.makedirs(gd_config_dir, exist_ok=True)
  os.makedirs(gd_output_dir, exist_ok=True)
  os.makedirs(gd_upscaled_dir, exist_ok=True)
  os.makedirs(gd_refine_dir, exist_ok=True)
  os.makedirs(gd_movie_dir, exist_ok=True)

  !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
  !pip install .
  !pip install .[stylize]
  !pip install xformers mediapipe
  !apt-get -qq install aria2
  os.makedirs(lora_dir, exist_ok=True)

model_path = f"models/sd/{model_name}.safetensors"
gd_custom_model_path = gd_model_dir + f"/{custom_model_name}.safetensors"
if model_name == "Custom":
  if not os.path.exists(gd_custom_model_path):
    raise ValueError("カスタムモデルがありません。{gd_custom_model_path}")
  model_path = f"models/sd/{custom_model_name}.safetensors"

def ignoreExists(dest, paths):
  result = []
  for path in paths:
    if os.path.isfile(os.path.join(dest, path)):
      result += path
  return result

# Model
model_file_path = os.path.join(data_dir, model_path)
if not os.path.exists(model_file_path):
  if model_name == "Custom":
    print(f"Google Driveのモデルを使用します。{gd_custom_model_path}")
    !cp {gd_custom_model_path} {model_file_path}
  else:
    model_urls = {
        "mistoonAnime_v20": "https://civitai.com/api/download/models/108545",
        "nadenadesitai_v10": "https://civitai.com/api/download/models/84669",
        "onigiriMix_v10": "https://civitai.com/api/download/models/163077",
        "surisurisitai_v10": "https://civitai.com/api/download/models/88061",
        "xxmix9realistic_v40": "https://civitai.com/api/download/models/102222"
    }
    print(f"モデルをダウンロードします。{model_name}: {model_urls[model_name]}")
    !aria2c {model_urls[model_name]} -o {model_file_path}

# MotionModule
motion_module_path = f"models/motion-module/{motion_module}.ckpt"
motion_module_file_path = os.path.join(data_dir, motion_module_path)
if not os.path.exists(motion_module_file_path):
  url = f"https://huggingface.co/guoyww/animatediff/resolve/main/{motion_module}.ckpt"
  !aria2c {url} -o {motion_module_file_path}

# LoRA
if lora_map != "":
  shutil.copytree(gd_lora_dir, lora_dir, ignore = ignoreExists, dirs_exist_ok = True)

# Emdeddings
shutil.copytree(gd_embeddings_dir, embeddings_dir, ignore = ignoreExists, dirs_exist_ok = True)

print(f"初期化: {(time.time() - initialize_start_time):.0f}秒")

# ------------------------------------------------------------------------------
# Generate movie
output_dir = "output"
upscaled_dir = "upscaled"
rifine_dir = "refine"

def get_new_dir(parent_dir):
  sub_dirs = []
  for path in os.listdir(parent_dir):
    dir_path = os.path.join(parent_dir, path)
    if os.path.isdir(dir_path):
      sub_dirs.append(dir_path)
  return max(sub_dirs, key = os.path.getmtime)

def get_first_dir(parent_dir):
  for p in os.listdir(parent_dir):
    path = os.path.join(parent_dir, p)
    if os.path.isdir(path):
      return path

def get_movie_path(parent_dir):
  for root, dirs, files in os.walk(parent_dir):
    for file in files:
      if file.endswith(".mp4"):
        return os.path.join(root, file)

def save_movie(parent_dir):
  movie_path = get_movie_path(parent_dir)
  if movie_download:
    files.download(movie_path)
  if output_to_google_drive != "Disable":
    shutil.copy(movie_path, gd_movie_dir + "/")

def encode_movie(parent_dir):
  frames_dir = get_first_dir(parent_dir)
  out_file = frames_dir + ".mp4"
  print(f"encode_movie:\n{frames_dir}\n{out_file}")
  input_pattern = f"{frames_dir}/%08d.png"
  !ffmpeg -y -framerate {output_fps} -i {input_pattern} -pix_fmt yuv420p -vcodec libx264 -r {output_fps} -crf {output_crf} {out_file}

for i in range(count):
  # Config
  mmddhhmmss = datetime.now().astimezone(timezone("Asia/Tokyo")).strftime("%m%d_%H%M_%S")
  config_file_path = f"{gd_config_dir}/Config-{mmddhhmmss}.json"
  config_txt = generate_config(mmddhhmmss)
  with open(config_file_path, "w") as file:
    file.write(config_txt)

  # Generate
  generate_start_time = time.time()
  print(f"--\nanimatediff generate -c {config_file_path} -W {width} -H {height} -L {length} -C {context}\n{config_txt}--")
  gc.collect()
  !animatediff generate -c {config_file_path} -W {width} -H {height} -L {length} -C {context}
  result_dir = get_new_dir(output_dir)
  print(f"生成: {(time.time() - generate_start_time):.0f}秒 {result_dir}");

  if output_to_google_drive == "All":
    dest_dir = os.path.join(gd_output_dir, os.path.basename(result_dir))
    shutil.copytree(result_dir, dest_dir, dirs_exist_ok = True)

  if upscale_mode == "Disable":
    save_movie(result_dir)
    continue

  # Upscale
  upscale_start_time = time.time()
  frames_dir = get_first_dir(result_dir)
  gc.collect()
  if upscale_mode == "Tile":
    print(f"--\nanimatediff tile-upscale {frames_dir} -W {upscale_width}")
    !animatediff tile-upscale {frames_dir} -W {upscale_width}
    result_dir = get_new_dir(upscaled_dir)
    encode_movie(result_dir)
    if output_to_google_drive == "All":
      dest_dir = os.path.join(gd_upscaled_dir, os.path.basename(result_dir))
      shutil.copytree(result_dir, dest_dir, dirs_exist_ok = True)
  else:
    print(f"--\nanimatediff refine {frames_dir} -W {upscale_width}")
    !animatediff refine {frames_dir} -W {upscale_width}
    result_dir = get_new_dir(rifine_dir)
    if output_to_google_drive == "All":
      dest_dir = os.path.join(gd_refine_dir, os.path.basename(result_dir))
      shutil.copytree(result_dir, dest_dir, dirs_exist_ok = True)
  save_movie(result_dir)
  print(f"アップスケール: {(time.time() - upscale_start_time):.0f}秒 {result_dir}");

print(f"総時間: {(time.time() - total_start_time):.0f}秒")
